<a href="https://colab.research.google.com/github/cryugesh01/Text-Summarization/blob/main/Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**EXTRACTIVE SUMMARIZATION**

In [ ]:
!pip install Pypdf
! pip install rouge

In [ ]:
import pypdf
from pypdf import PdfReader
import os

In [ ]:
def extract_text_from_pdf(pdf_path):
    text = ""
    reader = PdfReader(pdf_path)
    num_pages = len(reader.pages)
    for page_num in range(num_pages):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text
pdf_path='/content/drive/MyDrive/GLM/budget_speech (1).pdf'
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text[800:1000])

  
1. The Indian  economy  has witnessed profound positive 
transformation in the last ten years. The people of India are 
looking ahead to the future with hope and optimism.  
2. With the blessings o


TEXT PREPROCESSING

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from collections import Counter

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#lower case conversion
text=extracted_text.lower()
data=text[801:36100]


In [ ]:
# special charecter removal
def clean_text(sentences):
    cleaned_sentences = []
    for sentence in sentences:
        # Remove newlines
        sentence = sentence.replace('\n', ' ')

        # Remove serial numbers (e.g., "1. ", "2. ", etc.)
        words = sentence.split()
        cleaned_words = []
        for word in words:
            if  word[:-1].isdigit() and len(word) < 4:
                continue
            cleaned_words.append(word)
        cleaned_sentence = ' '.join(cleaned_words)
        cleaned_sentences.append(cleaned_sentence)

    return cleaned_sentences

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from heapq import nlargest
# Summary generator
def generate_summary(text, n):
  # sentence tokenization
  sentences = sent_tokenize(text)
  # Punctuation and special charecters removal
  cleaned_text = clean_text(sentences)

  # Create the TF-IDF matrix
  vectorizer = TfidfVectorizer(stop_words='english')
  tfidf_matrix = vectorizer.fit_transform(cleaned_text)

  # cosine similarity between each sentence and the document
  sentence_scores = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])[0]

  # Select the top n sentences with the highest scores
  summary_sentences = nlargest(n, range(len(sentence_scores)), key=sentence_scores.__getitem__)

  summary_tfidf = ' '.join([cleaned_text[i] for i in sorted(summary_sentences)])

  return summary_tfidf

In [ ]:
summary = generate_summary(data, 20)
summary_sentences = summary.split('. ')
formatted_summary = '.\n'.join(summary_sentences)

print(formatted_summary)

with the blessings of the people, when our government under the visionary and dynamic leadership of hon’ble prime minister shri narendra modi assumed office in 2014, the country was facing enormous challenges.
we expect that our government , based on its stupendous work , will be blessed ag ain by the people with a resounding mandate.
we are working to make india a ‘viksit bharat’ by 2047.
for achieving that goal, we need to improve people’s capability and empower them.
for our gov ernment, social justice is an effective and necessary governance model.
the saturation approach of covering all eligible people is the true and comprehensive achievement of social justice.
garib kalyan, desh ka kalyan we believe in empowering the poor.
‘direct benefit transfer’ of ` lakh crore from the government using pm -jan dhan accounts has led to savings of ` 2.7 lakh crore for the government .
the savings have helped in providing more funds for ‘garib kalyan’.
crop insurance is given to 4 crore farmers

In [ ]:
reference_summary="The Indian government, led by Prime Minister Shri Narendra Modi, aims to make India a viksit bharat by 2047 by improving people's capabilities and empowering them. Social justice is an effective governance model, and the government has implemented various initiatives to achieve this goal. These include direct benefit transfers, crop insurance, and mudra yojana loans. The government has a strong track record of governance, development, and performance, providing transparent, accountable, and people-centric administration. The vision for 'viksit bharat' is a prosperous bharat in harmony with nature, modern infrastructure, and providing opportunities for all citizens and regions to reach their potential. The government is also working on reforms in the states to achieve this vision. A committee will be mandated to make recommendations for addressing challenges comprehensively."

In [ ]:
# Evaluation
import rouge
from rouge import Rouge
def evaluate_rouge(reference_text, summary_text):
  rouge = Rouge()
  scores = rouge.get_scores(reference_text, summary_text)
  return scores[0]['rouge-1']['f']

In [ ]:
summary = generate_summary(data, 20)

# Evaluate the summary using ROUGE
rouge_score = evaluate_rouge(reference_summary, summary)

print(f"ROUGE score: {rouge_score}")

ROUGE score: 0.36206896157418417


**ABSTRACTIVE SUMMARIZATION**

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
model=T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer=T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
text1=data[801:4000]

In [ ]:
input=tokenizer.encode(text1,return_tensors='pt',max_length=512,truncation=True)

In [ ]:
input=tokenizer.encode(data,return_tensors='pt',max_length=512,truncation=True)

In [ ]:
output=model.generate(input,max_length=150,num_beams=7,early_stopping=True)

In [ ]:
output_text=tokenizer.decode(output[0],skip_special_tokens=True)
print(output_text)

the people blessed the government wi th a resounding mandate. 2 2. our government doubled down on its responsibilities to build a prosperous country with comprehensive development of all people and all regions. t he country overcame the challenge of a once -in-a-century pandemic, committed to ‘panch pran’.


In [ ]:
summary = generate_summary(data, 20)

# Evaluate the summary using ROUGE
rouge_score = evaluate_rouge(output_text, summary)

print(f"ROUGE score: {rouge_score}")

ROUGE score: 0.13745704245391532
